# Equity Listed Vanilla Option Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-eti) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [1]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
# IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DEKSTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.edp.thomsonreuters.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Computing the implied volatility and the greeks for a Listed Vanilla Option

This sample compute the implied volatility and the main greeks for a listed options. 'AAPLF182113000.U' is used and may have to be changed to another listed option code when it has expired. The returned 'fields' are:
- the option term & conditions
    (InstrumentCode,ExerciseType,OptionType,ExerciseStyle,EndDate,StrikePrice)
- the market value in cash (MarketValueInDealCcy)
- the implied volatility (VolatilityPercent)
- the main Greeks (DeltaPercent, GammaPercent, RhoPercent, ThetaPercent)
- the error information (ErrorCode,ErrorMessage)

the full documentation of the parameters available for equity options can be found in the [Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-eti)


In [2]:
request_body = {
    
    "fields": ["InstrumentCode","ExerciseType","OptionType","ExerciseStyle","EndDate","StrikePrice",
               "MarketValueInDealCcy","VolatilityPercent",
               "DeltaPercent","GammaPercent","ThetaPercent","ThetaPercent",
               "ErrorMessage"],
    
    "universe":[{
        "instrumentType": "Option",
        "instrumentDefinition": {
            "underlyingType": "Eti",
            "instrumentCode": "AAPLF182113000.U"
        }
    }],     
    
    "pricingParameters": {
        "valuationDate": "2021-01-12T00:00:00Z",
        "VolatilityType":"Implied"
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentCode'}, {'type': 'String', 'name': 'ExerciseType'}, {'type': 'String', 'name': 'OptionType'}, {'type': 'String', 'name': 'ExerciseStyle'}, {'type': 'DateTime', 'name': 'EndDate'}, {'type': 'Float', 'name': 'StrikePrice'}, {'type': 'Float', 'name': 'MarketValueInDealCcy'}, {'type': 'Float', 'name': 'VolatilityPercent'}, {'type': 'Float', 'name': 'DeltaPercent'}, {'type': 'Float', 'name': 'GammaPercent'}, {'type': 'Float', 'name': 'ThetaPercent'}, {'type': 'Float', 'name': 'ThetaPercent'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['AAPLF182113000.U', 'CALL', 'Vanilla', 'AMER', '2021-06-18T00:00:00Z', 130.0, 12.5, 39.1863136571643, 0.532472274987722, 0.0119754641554741, -0.0409053391267003, -0.0409053391267003, '']]}


In [3]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentCode,ExerciseType,OptionType,ExerciseStyle,EndDate,StrikePrice,MarketValueInDealCcy,VolatilityPercent,DeltaPercent,GammaPercent,ThetaPercent,ThetaPercent,ErrorMessage
0,AAPLF182113000.U,CALL,Vanilla,AMER,2021-06-18T00:00:00Z,130.0,12.5,39.186314,0.532472,0.011975,-0.040905,-0.040905,


2021-01-19 09:51:11,452 - Session session.platform - Thread 4756 | MainThread
[Error 400 - invalid_grant] empty error description
